In [1]:
import sys
sys.path.append("/home/dquintero/venvs/serviceDSSAT/lib/python3.10/site-packages")
sys.path.append('/home/dquintero/dev_dssat_service/dssatservice/')

In [11]:
!pwd

/home/dquintero/dssat_service/web_service_dev/demo_notebooks


In [2]:
from database import connect
from dssat import run_spatial_dssat
from ui.base import SCHEMAS, admin_list, AdminBase, BASELINE_YEARS, Session
from ui.plot import (
    validation_chart, init_anomalies_chart, add_anomaly_bar, clear_anomaly_chart,
    init_stress_chart, add_stress_bar, clear_stress_chart, init_columnRange_chart,
    add_yield_column
)
from data.transform import parse_overview

In [3]:
from datetime import datetime
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
DBNAME = "dssatserv"
con = connect(DBNAME)

In [7]:
selector_pars ={"schema": "kenya", "admin1": None, "session": None}
schema = selector_pars["schema"]

# session = Session(AdminBase(con, schema, "Bomet"))

admin1_list = admin_list(con, schema)

output_validation = widgets.Output()
def on_admin_change(change, session):
    selector_pars["session"] = Session(
        AdminBase(con, schema, change.new)
    )
    desc = selector_pars["session"].adminBase.baseline_description() + \
        ". If the assumptions are not accurate you can set your own baseline scenario."
    baseline_desc.value = f'<h4 style="padding: 0px 0px 0px 20px">{desc}</h4>'
    with output_validation:
        clear_output()
        val_chart = validation_chart(selector_pars["session"])
        val_chart.display()

admin1_picker = widgets.Dropdown(
    options=admin1_list,
    value=None,
    description='Admin1:',
    disabled=False,
)
admin1_picker.observe(lambda x: on_admin_change(x, selector_pars["schema"]), names='value')

baseline_desc = widgets.HTML(
    value=""
)


disp_box = widgets.VBox([
    widgets.HBox([admin1_picker, baseline_desc]),
    output_validation
])


selector_pars["session"]
display(disp_box)
# print(pars["adminBase"].baseline_description())

In [9]:
# selected_plot = "Anomaly Prob."
session = selector_pars["session"]
def change_plot_type(change):
    with output_yield:
        clear_output()
        if plot_selector.value == "Yield Range":
            range_chart.display()
        else:
            anom_chart.display()

plot_selector = widgets.RadioButtons(
    options=["Yield Range", "Anomaly Prob."],
    description='Chart:',
    disabled=False,
    value="Anomaly Prob."
)
plot_selector.observe(change_plot_type, names="value")

def on_value_change(change, par):
    if par == "cultivar":
        session.simPars.__dict__[par] = session.adminBase.cultivar_labels_inv[change.new]
    else:
        session.simPars.__dict__[par] = change.new
        
plantingdate_picker = widgets.DatePicker(
    description='Planting date:',
    disabled=False,
    value=session.simPars.planting_date
)
plantingdate_picker.observe(lambda x: on_value_change(x, "planting_date"), names='value')

cultivar_picker = widgets.Dropdown(
    options=session.adminBase.cultivar_labels.values(),
    description='Cultivar:',
    disabled=False,
    value=session.adminBase.cultivar_labels[session.simPars.cultivar]
)
cultivar_picker.observe(lambda x: on_value_change(x, "cultivar"), names='value')

run_button = widgets.Button(
    description='Run DSSAT',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run DSSAT',
    layout=widgets.Layout(width='95%',)
)
clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear',
    layout=widgets.Layout(width='95%',)
)

baseline_button = widgets.Button(
    description='Change Baseline',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Set Baseline',
    layout=widgets.Layout(width='95%',)
)

nitrogen_dap = widgets.IntsInput(
    value=session.simPars.nitrogen_dap,
    min=0,
    max=120,
    format='d',
    description='N date(dap):'
)
nitrogen_dap.observe(lambda x: on_value_change(x, "nitrogen_dap"), names='value')

nitrogen_rate = widgets.IntsInput(
    value=session.simPars.nitrogen_rate,
    min=0,
    max=999,
    format='d',
    description='N rate:'
)
nitrogen_rate.observe(lambda x: on_value_change(x, "nitrogen_rate"), names='value')

irrigation = widgets.Checkbox(
    value=False,
    description='Irrigation',
    disabled=True,
    indent=False
)

output_yield = widgets.Output()
output_stress1 = widgets.Output()
output_stress2 = widgets.Output()

select_box = widgets.VBox([
    widgets.HBox([
        widgets.HBox([
            widgets.VBox([run_button, clear_button, baseline_button]),
            plot_selector
        ]),
        widgets.HBox([
            widgets.VBox([plantingdate_picker, cultivar_picker]),
            widgets.VBox([
                widgets.HBox([widgets.HTML(value="Timing (dap):"), nitrogen_dap]),
                widgets.HBox([widgets.HTML(value="Rate (kg N/ha):"), nitrogen_rate]),
            ])
        ]),
    ]),
    
    output_yield,
    widgets.HBox([
        widgets.HBox([output_stress1,]),
        widgets.HBox([output_stress2,])
    ])
])

anom_chart = init_anomalies_chart()
range_chart = init_columnRange_chart()
stress_chart_1 = init_stress_chart("water")
stress_chart_2 = init_stress_chart("nitrogen")

def clear_figure(b):
    clear_anomaly_chart(anom_chart)
    clear_anomaly_chart(range_chart)
    clear_stress_chart(stress_chart_1)
    clear_stress_chart(stress_chart_2)
    with output_yield:
        if plot_selector.value == "Yield Range":
            range_chart.display()
        else:
            anom_chart.display()
    with output_stress1:
        stress_chart_1.display()
    with output_stress2:
        stress_chart_2.display()

def on_clic_run(b):
    session.run_experiment()
    with output_yield:
        add_anomaly_bar(anom_chart, session)
        add_yield_column(range_chart, session)
        clear_output()
        if plot_selector.value == "Yield Range":
            range_chart.display()
        else:
            anom_chart.display()
    with output_stress1:
        add_stress_bar(stress_chart_1, session)
        clear_output()
        stress_chart_1.display()
    with output_stress2:
        add_stress_bar(stress_chart_2, session)
        clear_output()
        stress_chart_2.display()
    n_exp = len(stress_chart_1.options.series)
        
def on_clic_baseline(b):
    session.new_baseline()
    
clear_button.on_click(clear_figure)
run_button.on_click(on_clic_run)
baseline_button.on_click(on_clic_baseline)
display(select_box, output_yield, output_stress1, output_stress2)

Output()

Output()

Output()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  9.59it/s]


In [13]:
session.adminBase.baseline_stats

,mean,std
quantile,,
0.025,0.238870,0.097385
0.075,0.322600,0.105744
0.125,0.383375,0.086620
0.175,0.425535,0.094534
0.225,0.490260,0.114907
0.275,0.549625,0.157314
0.325,0.594205,0.144332
0.375,0.627150,0.144356
0.425,0.670290,0.164306


In [12]:
session.adminBase.baseline_pars

SimulationPars(nitrogen_dap=[5, 30, 60], nitrogen_rate=[6, 6, 15], cultivar='IF0010', planting_date=datetime.date(2018, 5, 1))

In [19]:
session.experiment_results

,planting,nitro_dap,nitro_rate,cultivar,yield_range,harvest_range
0,2018-04-01,"(5, 30, 60)","(6.018470460529408, 6.018470460529408, 6.01847...",IF0010,"(0.34445000000000003, 1.8898)","(83, 91)"
1,2019-04-01,"(5, 30, 60)","(6.018470460529408, 6.018470460529408, 6.01847...",IF0010,"(0.3012, 1.3212)","(77, 86)"
2,2020-04-01,"(5, 30, 60)","(6.018470460529408, 6.018470460529408, 6.01847...",IF0010,"(0.3063, 1.7209999999999996)","(81, 91)"
3,2021-04-01,"(5, 30, 60)","(6.018470460529408, 6.018470460529408, 6.01847...",IF0010,"(0.18370000000000003, 1.61395)","(81, 89)"
4,2022-04-01,"(5, 30, 60)","(6.018470460529408, 6.018470460529408, 6.01847...",IF0010,"(0.2151, 1.3489)","(82, 90)"
5,2022-04-01,"(5, 30, 60)","(6.018470460529408, 6.018470460529408, 6.01847...",IF0007,"(0.3389, 1.73955)","(106, 117)"


In [9]:
session.latest_overview

['*SIMULATION OVERVIEW FILE\n',
 '\n',
 '*DSSAT Cropping System Model Ver. 4.8.2.000 -release  MAY 30, 2024 20:43:07\n',
 '\n',
 '*RUN   1        : SAMPLE 1                  MZCER048 EXPEFILE    1\n',
 ' MODEL          : MZCER048 - Maize\n',
 ' EXPERIMENT     : EXPEFILE MZ SPATIAL ANALYSES TEST CASE; FLORENCE, SOUTH CAROLI\n',
 ' DATA PATH      : /tmp/dssatrun1UFA6LBP/\n',
 ' TREATMENT  1   : SAMPLE 1                  MZCER048\n',
 '\n',
 '\n',
 ' CROP           : Maize            CULTIVAR : TZEEY-SRBC5      ECOTYPE :IB0001\n',
 ' STARTING DATE  : JUN  1 2018\n',
 ' PLANTING DATE  : AUTOMATIC PLANTING PLANTS/m2 :  4.0     ROW SPACING :  90.cm\n',
 ' WEATHER        : SEAA   2018\n',
 ' SOIL           : IB00000001     TEXTURE : yLoam -    ISRIC soilgrids + HC27\n',
 ' SOIL INIT COND : DEPTH:200cm EXTR. H2O:255.4mm  NO3:  0.0kg/ha  NH4:  0.0kg/ha\n',
 ' WATER BALANCE  : RAINFED\n',
 ' IRRIGATION     : NOT IRRIGATED\n',
 ' NITROGEN BAL.  : SOIL-N & N-UPTAKE SIMULATION; NO N-FIXATION\n',
 '

In [24]:
session.latest_run

,RUN,CR,TRT,FLO,MAT,TOPWT,HARWT,RAIN,TIRR,CET,PESW,TNUP,TNLF,TSON,TSOC,year,sim
0,1,MZ,1,99,177,5144,2760,1326,0,702,281,69,36,22877,229,1,2.760
1,2,MZ,2,87,149,4043,1863,773,0,525,202,48,25,12124,127,1,1.863
2,3,MZ,3,85,144,4659,2102,624,0,500,205,54,29,15383,160,1,2.102
3,4,MZ,4,91,156,5420,2759,765,0,567,225,66,35,18548,189,1,2.759
4,5,MZ,5,86,147,6809,3448,443,0,498,175,82,31,23086,234,1,3.448
5,6,MZ,6,90,154,4395,2088,765,0,551,237,52,32,15929,162,1,2.088
6,7,MZ,7,112,210,4026,2167,1560,0,785,290,60,30,18544,189,1,2.167
7,8,MZ,8,98,173,5478,2931,824,0,585,201,73,34,20430,206,1,2.931
8,9,MZ,9,86,147,5139,2326,443,0,468,180,60,30,14833,153,1,2.326
9,10,MZ,10,95,167,4535,2340,810,0,557,217,57,32,16725,171,1,2.340
